In [8]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext ,SQLContext,Row,HiveContext
import sys
import re
from pyspark.sql import Row
from pyspark.sql.types import *

conf = SparkConf().setAppName("Apache Log Analyzer")
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

s='64.242.88.10 - - [07/Mar/2004:16:05:49 -0800] "GET /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1" 401 12846'
print (s)

64.242.88.10 - - [07/Mar/2004:16:05:49 -0800] "GET /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1" 401 12846


In [3]:
'''
Creating a function to read each element using Regex. This module will be passed later on in the notebook.
'''
def log_line(line):
    patrn = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+)'
    mtch = re.search(patrn,line)
    if mtch is None:
        pass
    else :
        ip = mtch.group(1)
        client = mtch.group(2)
        user = mtch.group(3)
        date_time = mtch.group(4)
        mode = mtch.group(5)
        target = mtch .group(6)
        protocol = mtch.group(7)
        res_code = mtch.group(8)
        size = mtch.group(9)
        return Row(ip,client,user,date_time,mode,target,protocol,res_code,size)
print (log_line(s))

<Row(64.242.88.10, -, -, 07/Mar/2004:16:05:49 -0800, GET, /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables, HTTP/1.1, 401, 12846)>


In [21]:
# Reading source file form HDFS and creating RDD

logFileRdd = sc.textFile("/user/data/apache_log.txt")
logFileRdd.take(5)
parseLogFileRdd = logFileRdd.map(log_line).cache()
parseLogFileRdd.take(5)

[<Row(64.242.88.10, -, -, 07/Mar/2004:16:05:49 -0800, GET, /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables, HTTP/1.1, 401, 12846)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:06:51 -0800, GET, /twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2, HTTP/1.1, 200, 4523)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:10:02 -0800, GET, /mailman/listinfo/hsdivision, HTTP/1.1, 200, 6291)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:11:58 -0800, GET, /twiki/bin/view/TWiki/WikiSyntax, HTTP/1.1, 200, 7352)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:20:55 -0800, GET, /twiki/bin/view/Main/DCCAndPostFix, HTTP/1.1, 200, 5253)>]

In [19]:
def build_schema():
    schema = StructType(
       [
        StructField("ip",          StringType(),  False),
        StructField("client",      StringType(),  False),
        StructField("user",        StringType(),  False),
        StructField("date_time",   StringType(),  False),
        StructField("mode",        StringType(),  False),
        StructField("target",      StringType(),  False),
        StructField("protocol",    StringType(),  False),
        StructField("res_code",    StringType(),  False),
        StructField("size",        StringType(),  False)
       ]
   )
    return schema


In [32]:
#hist_data = sqlContext.sql()
dataDF = sqlContext.createDataFrame(parseLogFileRdd, schema=build_schema()).cache()
dataDF.show()

+------------+------+----+--------------------+----+--------------------+--------+--------+-----+
|          ip|client|user|           date_time|mode|              target|protocol|res_code| size|
+------------+------+----+--------------------+----+--------------------+--------+--------+-----+
|64.242.88.10|     -|   -|07/Mar/2004:16:05...| GET|/twiki/bin/edit/M...|HTTP/1.1|     401|12846|
|64.242.88.10|     -|   -|07/Mar/2004:16:06...| GET|/twiki/bin/rdiff/...|HTTP/1.1|     200| 4523|
|64.242.88.10|     -|   -|07/Mar/2004:16:10...| GET|/mailman/listinfo...|HTTP/1.1|     200| 6291|
|64.242.88.10|     -|   -|07/Mar/2004:16:11...| GET|/twiki/bin/view/T...|HTTP/1.1|     200| 7352|
|64.242.88.10|     -|   -|07/Mar/2004:16:20...| GET|/twiki/bin/view/M...|HTTP/1.1|     200| 5253|
|64.242.88.10|     -|   -|07/Mar/2004:16:23...| GET|/twiki/bin/oops/T...|HTTP/1.1|     200|11382|
|64.242.88.10|     -|   -|07/Mar/2004:16:24...| GET|/twiki/bin/view/M...|HTTP/1.1|     200| 4924|
|64.242.88.10|     -

In [33]:
from pyspark.sql.functions import *
resCodeCntDf = dataDF.select('res_code').groupBy('res_code').count().orderBy(desc("count"))
resCodeCntDf.show()

+--------+-----+
|res_code|count|
+--------+-----+
|     200| 1272|
|     401|  123|
|     302|    6|
|     404|    5|
+--------+-----+



In [36]:
# Calculate  content size .
from pyspark.sql import functions as F
contenDf = dataDF.select('size').agg(F.avg(dataDF.size).alias("avg_content"),F.max(dataDF.size).alias("max_content"),
                                     F.min(dataDF.size).alias("min_content"))
contenDf.show()

+-----------------+-----------+-----------+
|      avg_content|max_content|min_content|
+-----------------+-----------+-----------+
|7775.963726884779|       9812|          0|
+-----------------+-----------+-----------+



In [40]:
ipDf = dataDF.select('ip').groupBy('ip').count().orderBy(desc("count")).where(col('count') >20)
ipDf.show()

+--------------------+-----+
|                  ip|count|
+--------------------+-----+
|        64.242.88.10|  452|
|          10.0.0.153|  188|
|cr020r01-3.sac.ov...|   44|
|h24-71-236-129.ca...|   36|
|h24-70-69-74.ca.s...|   32|
|market-mail.pandu...|   29|
|ts04-ip92.hevanet...|   28|
|ip68-228-43-49.tc...|   22|
+--------------------+-----+

